# 文部科学省 子供の学習費調査データ分析サンプル

このドキュメントでは、文部科学省の子供の学習費調査データを取得・処理・分析する方法を示します。

## データソース
- 文部科学省 子供の学習費調査
- e-Stat 令和５年度 子供の学習費調査
- URL: https://www.e-stat.go.jp/stat-search/files?page=1&layout=datalist&toukei=00400201&tstat=000001012023&cycle=0&tclass1=000001224200&tclass2=000001224201&tclass3=000001224327&tclass4val=0

## 1. ライブラリのインポート

In [2]:
import sys
import os

# 必要なライブラリのインポート
import pandas as pd
import numpy as np

# 表示設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

In [5]:
# パッケージのパスを追加（exampleフォルダから一つ上の階層のfpy_datareaderを参照）
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'fpy_datareader'))

## 2. child_educational_expenses モジュールのインポート

In [6]:
# 学習費データ処理モジュールのインポート
from child_educational_expenses import (
    get_processed_tuition_data,
    create_tuition_pivot_table,
    load_tuition_education_data,
    load_tuition_mapping,
    process_tuition_mapping
)

## 3. 基本的なデータ取得と確認

### 3.1 カテゴリマッピングの確認

In [ ]:
# カテゴリマッピングテーブルの読み込みと確認
print("=== カテゴリマッピングテーブルの読み込み ===")
raw_mapping = load_tuition_mapping()
processed_mapping = process_tuition_mapping(raw_mapping)

print(f"\nマッピングテーブルの形状: {processed_mapping.shape}")
print("\nマッピングテーブルの内容（最初の10行）:")
display(processed_mapping.head(10))

print("\n=== 学習費区分の階層構造 ===")
print("レベル1（最上位）:", processed_mapping['level1'].unique())
print("レベル2:", processed_mapping['level2'].unique())
print("レベル3:", processed_mapping['level3'].unique())

### 3.2 生データの確認

In [ ]:
# 生データの読み込み（時間がかかる可能性があります）
print("=== 生データの読み込み ===")
print("注意: この処理には数分かかる場合があります")

education_data = load_tuition_education_data()

print("\n=== 各教育段階のデータ形状 ===")
for level, df in education_data.items():
    print(f"{level}: {df.shape}")

print("\n=== 小学校データのサンプル ===")
print("列名（MultiIndex）:")
display(education_data['elementary'].columns.tolist()[:5])  # 最初の5列

print("\nインデックス（学習費区分）:")
print(education_data['elementary'].index.tolist()[:10])  # 最初の10項目

print("\nデータサンプル（最初の5行5列）:")
display(education_data['elementary'].iloc[:5, :5])

## 4. 処理済みデータの取得と基本分析

### 4.1 完全処理済みデータの取得

In [ ]:
# 完全に処理されたデータを取得
print("=== 完全処理済みデータの取得 ===")
print("注意: この処理には数分かかる場合があります")

df_final = get_processed_tuition_data()

print(f"\n最終データセットの形状: {df_final.shape}")
print(f"カラム: {list(df_final.columns)}")

print("\n=== データの基本情報 ===")
print(df_final.info())

print("\n=== 各カテゴリの件数 ===")
print(f"公立・私立区分: {df_final['公立・私立区分'].value_counts()}")
print(f"\n学校種別: {df_final['学校'].value_counts()}")
print(f"\n年齢範囲: {df_final['年齢'].min()} - {df_final['年齢'].max()}歳")

### 4.2 データサンプルの表示

In [ ]:
# データサンプルの表示
print("=== 処理済みデータのサンプル ===")
display(df_final.head(50))

print("\n=== 欠損値の確認 ===")
print(df_final.isnull().sum())

## 5. ピボットテーブルの作成と分析

### 5.1 ピボットテーブルの作成


In [ ]:
# ピボットテーブルの作成
print("=== ピボットテーブルの作成 ===")
pivot_table = create_tuition_pivot_table(df_final)

print(f"ピボットテーブルの形状: {pivot_table.shape}")
print(f"インデックス名: {pivot_table.index.names}")
print(f"カラム階層数: {len(pivot_table.columns.levels)}")

print("\n=== ピボットテーブルのサンプル（最初の5行5列） ===")
display(pivot_table.iloc[:5, :5])

## 7. データエクスポートと保存

### 7.1 結果の保存

In [ ]:
# 結果をファイルに保存
def save_tuition_data(df, output_dir='output'):
   """
   学習費データをファイルに保存
   """
   import os
   
   # 出力ディレクトリの作成
   os.makedirs(output_dir, exist_ok=True)
   
   # 処理済みデータの保存
   df.to_csv(f'{output_dir}/tuition_data.csv', index=False, encoding='utf-8-sig')
   df.to_excel(f'{output_dir}/tuition_data.xlsx', index=False)
   
   print(f"データを{output_dir}ディレクトリに保存しました")
   print("保存されたファイル:")
   print("- processed_tuition_data.csv: 処理済みデータ(CSV)")
   print("- processed_tuition_data.xlsx: 処理済みデータ(Excel)")

# 結果の保存実行
save_tuition_data(df_final)